In [3]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from typing import TypedDict,Literal
from dotenv import load_dotenv
from pydantic import BaseModel,Field

load_dotenv()

model=ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_retries=2)



class SentimentSchema(BaseModel):
      sentiment: Literal["positive", "negative"] = Field(description='Sentiment of the review')

class DiagnosisSchema(BaseModel):
  issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
  tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
  urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

mode_1= model.with_structured_output(SentimentSchema)
mode_2= model.with_structured_output(DiagnosisSchema)



class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

def find_sentiment(state: ReviewState):
    prompt = f"""
    For the following review, determine the sentiment.
    Respond with one word only: positive or negative.

    Review:
    {state['review']}
    """
    
    result = mode_1.invoke(prompt)
    return {"sentiment": result.sentiment}

def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:
    
    if state['sentiment'] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"
    

def positive_response(state: ReviewState):
    prompt = f"""
    Write a warm thank-you message in response to this review:

    "{state['review']}"

    Also kindly ask the user to leave feedback on our website.
    """
    
    response = model.invoke(prompt).content
    return {"response": response}

def run_diagnosis(state: ReviewState):
    prompt = f"""
    Diagnose this negative review:

    {state['review']}

    Return:
    - issue_type
    - tone
    - urgency
    """
    
    diagnosis = mode_2.invoke(prompt)
    return {"diagnosis": diagnosis.model_dump()}

def negative_response(state: ReviewState):
    diagnosis = state['diagnosis']

    prompt = f"""
    You are a support assistant.

    The user had a '{diagnosis['issue_type']}' issue,
    sounded '{diagnosis['tone']}',
    and urgency was '{diagnosis['urgency']}'.

    Write an empathetic and helpful resolution message.
    """
    
    response = model.invoke(prompt).content
    return {"response": response}


graph= StateGraph(ReviewState)

graph.add_node('find_sentiment',find_sentiment)
graph.add_node('positive_response',positive_response)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)

graph.add_edge(START,'find_sentiment')
graph.add_conditional_edges('find_sentiment',check_sentiment)

graph.add_edge('positive_response',END)

graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response',END)

workflow= graph.compile()

intial_state={
    'review': "I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}
workflow.invoke(intial_state)

{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'angry', 'urgency': 'high'},
 'response': "I'm so sorry to hear that you're experiencing a bug issue and that it's causing you frustration. I can imagine how annoying it must be to deal with this, especially when you need a resolution as soon as possible.\n\nPlease know that I'm here to help and I'm committed to resolving this issue for you. I'm going to do my best to provide a prompt and effective solution. Can you please provide me with more details about the bug you're experiencing, such as any error messages you're seeing or the steps you took leading up to the issue? This will help me to better understand the problem and work towards a resolution.\n\nIn the meantime, I wa